<table style="width:100%">
  <tr>
    <td><center style="font-size:500%;">Representação Distribuida de Palavras (Parte 2)</center></td>
    <td><img src="https://logodownload.org/wp-content/uploads/2015/02/puc-rio-logo.gif" width="100"/></td> 
  </tr>    
</table>

Msc. Cristian Muñoz V.

In [0]:
%matplotlib inline
import glob
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import csv
import gensim
from itertools import groupby
from gensim.similarities import WmdSimilarity
from gensim.models import Word2Vec

nltk.download('punkt') 
nltk.download('stopwords')
stop_words = stopwords.words('english')

In [0]:
def read_files(filenames):
  lines = []
  # Lendo todos os documentos de texto
  for index,filename in enumerate(filenames):

      file = open(filename, 'r', encoding='utf-8')

      for line in file:

          # Remover espaços em branco 
          line = line.strip()

          # Pula linhas em branco
          if len(line) == 0:
              continue

          lines.append(line)

      if index % 10 ==0:
          print(".",end=" ")
  
  print("Letirua Completada!")
  
  text = " ".join(lines)
  
  return text

In [0]:
filenames =  glob.glob("PRH/*")

text = read_files(filenames)
text = nltk.word_tokenize(text , language='portuguese')
print("Tokenizando Completada!")

In [0]:
# Agrupamos por texto que se encontra entre 2 ponto.
sentences = [list(group) for k, group in groupby(text, lambda x: x == ".")\
                             if not k]

# Remove stopwords numbers and punctuation.

def preprocess(sentence):
  return [word for word in sentence\
                if word not in stop_words and word.isalpha()]

sentences = [preprocess(sentence) for sentence in sentences]

# Seleccionamos frases com dimensão entre 3 e 150 palavras.
sentences =  [sentence for sentence in sentences\
                        if len(sentence)>3 and len(sentence)<150]

In [0]:
lens = [len(sentence) for sentence in sentences]
avg_len = sum(lens) / float(len(lens))

plt.figure(figsize=(10,6))
plt.hist([len(sentence) for sentence in sentences])
plt.axvline(avg_len, color='#e41a1c')
plt.title('Histograma de cumprimento de frases.')
plt.xlabel('tamanho')
plt.text(10, 40000, 'mean = %.2f' % avg_len)
plt.show()

In [0]:
# Treinamos Word2Vec com todos as frases
model = Word2Vec(sentences, workers=2, size=50)

# Calculamos a similaridade entre um conjunto de frases
num_best = 10
instance = WmdSimilarity(sentences[:1000], model, num_best=10)

In [0]:
text = 'muito obrigado'
sentence = nltk.word_tokenize(text.lower(), language='portuguese')
query = preprocess(sentence)
sims = instance[query]  # A query observa na classe de similaridade.

In [0]:
# Mostramos os resultado à da frase perguntada na celula anterior 
print('Query:')
print(text)
for i in range(num_best):
    print("")
    print('sim = %.4f' % sims[i][1])
    print(" ".join(sentences[sims[i][0]]))

In [0]:
model.wv.most_similar('petróleo')

In [0]:
tensor_file = 'word_tensor.w2v'
model.wv.save_word2vec_format(tensor_file)
key_vectors = gensim.models.KeyedVectors.load_word2vec_format(tensor_file, binary=False)

In [0]:
words_id = []
vectors = []
for word_id in key_vectors.index2word[:10000]:
    words_id.append(word_id)
    vectors.append(key_vectors[word_id])

In [0]:
def create_tensorboard_files(tensor_filename, vectors, metadatos, colnames=None):

    out_file_tsv      = tensor_filename + '_tensor.tsv'
    out_file_tsv_meta = tensor_filename + '_metadata.tsv'
    
    with open(out_file_tsv, 'w',encoding='utf-8') as f:   
        for vector in vectors:
            vector_str = "\t".join([str(x) for x in vector])
            f.write( vector_str + '\n')

    with open(out_file_tsv_meta, 'w',encoding='utf-8') as f:
        writer = csv.writer(f, delimiter='\t')
        if len(metadatos)>=2:
            if colnames is None:
                colnames = "\t".join([str(i) for i in range(len(metadatos))])
            writer.writerow(colnames)
            for metadato in zip(*metadatos):
                line = [str(x) for x in metadato]
                writer.writerow(line)
        else:
            for metadato in metadatos[0]:
                writer.writerow([metadato])
            
    print("Arquivo com o Tensor 2D foi salvado em: %s" % out_file_tsv)
    print("Arquivo com o Tensor de metadatos foi salvado em: %s" % out_file_tsv_meta)

In [0]:
metadatos=[words_id]
create_tensorboard_files(tensor_filename="word", 
                      vectors=vectors, 
                      metadatos=[words_id])

Baixa os arquivos gerados e colocar em Tensoboard:


In [0]:
from google.colab import files
files.download('word_tensor.tsv')
files.download('word_metadata.tsv')